In [9]:
import csv
import numpy as np
import scipy
import networkx as nx
import math
import sys
import os
import subprocess
import scipy.stats
import scipy.optimize
import operator
import time
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import copy

sys.path.append("../Python_code") # go to parent dir
from canvas_vis import * 
from analytics_combined import *
from generate_proj_to_remove import *
from project_data_analysis import *
from segmentation import *
import pickle

In [12]:
class BboxIter:
    """
        Iterator for building boulding boxes for updates, one at a time
    """
    def __init__(self, input_file_name, dim, final=False, x_min=0, x_max=1000, y_min=0, y_max=1000,begin_time=0,end_time=sys.maxsize):
        '''
            @input_file_name is the input file with updates
            @dim is the height/width of bounding boxes in pixels
            @final is whether only updates in the final canvas should be considered
            @x_min, @x_max, @y_min, @max, @begin_time, @end_time constrain the set of udpates considered
        '''
        self.index = 1
        self.x0 = 0
        self.y0 = 0
        self.dim = dim
        self.final = final
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        
        self.file = open(input_file_name,'r')
        # Skip first line (header row)
        next(self.file)
        
        self.canvas = []
        
        for i in range(y_max-y_min+1):
            self.canvas.append([])
            
            for j in range(x_max-x_min+1):
                self.canvas[-1].append(None)

    def __iter__(self):
        return self

    def __next__(self):
        '''
            Returns list of bounding boxes associated to next update in the file.
            Each bounding box contains the following matrices (with same dimensions):
            ids_matrix: matrix with update IDs
            rgb_matrix: RGB matrix
            label_matrix: matrix with drawing labels
            user_matrix: matrix with user labels
        '''
        pixel_color = 0
        n = 0
        while (pixel_color == 0 and self.final is True) or n == 0:
            r = next(self.file).split(',')
            
            #ts,user,x_coordinate,y_coordinate,color,pic_id,pixel,pixel_color,proj_smallest
            ts = int(r[0])
            user = r[1]
            x = int(r[2])
            y = int(r[3])
            color = int(r[4])
            pic_id = int(r[5])
            pixel = int(r[6])
            pixel_color = int(r[7])
            proj_smallest = int(r[8])
            
            if proj_smallest == 1:
                self.canvas[y-self.y_min][x-self.x_min] = [self.index, ts, user, x, y, color, pic_id, pixel, pixel_color, proj_smallest]
                
                n = n + 1
                
            self.index = self.index + 1
        
        if n > 0:            
            x_min = max(int(math.ceil(x-self.dim/2)),0)
            x_max = min(int(math.ceil(x+self.dim/2)), self.x_max)
                            
            y_min = max(int(math.ceil(y-self.dim/2)),0)
            y_max = min(int(math.ceil(y+self.dim/2)), self.y_max)
            updates = []
            
            for xi in range(x_min, x_max):
                for yi in range(y_min,y_max): 
                    if self.canvas[yi][xi] != None:
                        updates.append(self.canvas[yi][xi])
                                        
            rgb_matrix = build_rgb_matrix(updates, x_min, y_min, self.dim)
            label_matrix, pic_ids = build_label_matrix(updates, x_min, y_min, self.dim)
            user_matrix = build_user_matrix(updates, x_min, y_min, self.dim)
            ids_matrix = build_update_id_matrix(updates, x_min, y_min, self.dim)
                            
            return [ids_matrix, rgb_matrix, label_matrix, pic_ids, user_matrix], self.index
                    
def build_rgb_matrix(updates, x0, y0, dim):
    """
    """
    min_x=1001
    min_y=1001
    
    for up in updates:
        x = up[3]
        y = up[4]

        min_x = min(x,min_x)
        min_y = min(y,min_y)
    
    data_color_code = np.uint8(16 * np.ones((dim,dim)))
    
    for up in updates:
        x = up[3]
        y = up[4]
        color = up[5]

        data_color_code[y-min_y][x-min_x] = color

    return canvas_color_code_rgb(data_color_code)

def build_label_matrix(updates, x0, y0, dim):
    """
    """
    min_x=1001
    min_y=1001
    
    for up in updates:
        x = up[3]
        y = up[4]

        min_x = min(x,min_x)
        min_y = min(y,min_y)
    
    labels = np.uint16(np.zeros((dim,dim)))
    
    pic_id_map = {}
    pic_ids = []
    
    for up in updates:
        x = up[3]
        y = up[4]
        pic_id = up[6]
        pixel = up[7]

        if pixel == 1:
            if pic_id not in pic_id_map:
                pic_id_map[pic_id] = len(pic_id_map)+1
                pic_ids.append(pic_id)
                
            labels[y-min_y][x-min_x] = pic_id_map[pic_id]
            
    if np.min(labels) > 0:
        labels = labels-1
    else:
        pic_ids = [0]+pic_ids

    return labels, pic_ids

def build_user_matrix(updates, x0, y0, dim):
    """
    """
    min_x=1001
    min_y=1001
    
    for up in updates:
        x = up[3]
        y = up[4]

        min_x = min(x,min_x)
        min_y = min(y,min_y)
    
    users = np.array([None for item in range(dim*dim)], dtype=object).reshape(dim,dim)
    
    for up in updates:
        x = up[3]
        y = up[4]
        user = up[2]

        users[y-min_y][x-min_x] = user

    return users

def build_update_id_matrix(updates, x0, y0, dim):
    """
    """
    min_x=1001
    min_y=1001
    
    for up in updates:
        x = up[3]
        y = up[4]

        min_x = min(x,min_x)
        min_y = min(y,min_y)
    
    update_ids = np.uint32(np.zeros((dim,dim)))
    
    for up in updates:
        x = up[3]
        y = up[4]
        ID = up[0]

        update_ids[y-min_y][x-min_x] = ID

    return update_ids

In [ ]:
def draw_bbox(updates, output_file_name):
    min_x=1001
    max_x=0
    min_y=1001
    max_y=0
    
    for up in updates:
        x = up[3]
        y = up[4]

        min_x = min(x,min_x)
        min_y = min(y,min_y)
        max_x = max(x,max_x)
        max_y = max(y,max_y)
        
    
    data_color_code = np.uint8(np.zeros((max_y-min_y+1,max_x-min_x+1)))

    for up in updates:
        x = up[3]
        y = up[4]
        color = up[5]

        data_color_code[y-min_y][x-min_x] = color

    canvas_ups_rgb = canvas_color_code_rgb(data_color_code)

    draw_canvas(canvas_ups_rgb, output_file_name+".svg")


In [ ]:
#Saving bounding boxes to pickle files

n = 0
updates_per_file = 100000
f = 0
bboxes = []

#16 images per update
for b in BboxIter("../data/sorted_tile_placements_proj.csv", 32, final=False):
    bboxes.append(b)
    
    if (n+1) % updates_per_file == 0:
        pickle.dump(bboxes, open(output_file_name, "wb"))
        f = f+1
        output_file_name = "../data/train_frames_"+str(f)+".pckl"
        
        print("Saved ", output_file_name)
        bboxes = []
        
    n = n + 1


Saved  ../data/train_frames_1.pckl
Saved  ../data/train_frames_2.pckl
Saved  ../data/train_frames_3.pckl
Saved  ../data/train_frames_4.pckl
Saved  ../data/train_frames_5.pckl
Saved  ../data/train_frames_6.pckl
Saved  ../data/train_frames_7.pckl
Saved  ../data/train_frames_9.pckl
Saved  ../data/train_frames_10.pckl
Saved  ../data/train_frames_11.pckl
Saved  ../data/train_frames_12.pckl
Saved  ../data/train_frames_13.pckl
Saved  ../data/train_frames_14.pckl
Saved  ../data/train_frames_15.pckl
Saved  ../data/train_frames_16.pckl
Saved  ../data/train_frames_17.pckl
Saved  ../data/train_frames_18.pckl
Saved  ../data/train_frames_19.pckl
Saved  ../data/train_frames_20.pckl
Saved  ../data/train_frames_21.pckl
Saved  ../data/train_frames_22.pckl
Saved  ../data/train_frames_23.pckl


In [15]:
input_file_name = "../data/ntrain_frames_50.pckl"
bboxes = pickle.load(open(input_file_name, "rb"))

In [16]:
len(bboxes)

1000

In [17]:
bboxes[1]

[array([[2021985, 1790233, 1964313, ..., 2124465, 2175268,  156439],
        [1947598, 2032701, 1842376, ..., 2123419, 2157995,       0],
        [1994075, 2051525, 2094621, ..., 2403314, 2140132,       0],
        ...,
        [1461693, 1520185, 1534890, ..., 1762920,  121572,  133360],
        [1418975, 1472238, 1533965, ...,       0,       0,  138110],
        [1411538, 1941780, 1477678, ..., 1408640,       0,    3982]],
       dtype=uint32), array([[[255, 167, 209],
         [229,   0,   0],
         [229, 149,   0],
         ...,
         [229,   0,   0],
         [ 34,  34,  34],
         [130,   0, 128]],
 
        [[130,   0, 128],
         [255, 167, 209],
         [229,   0,   0],
         ...,
         [229,   0,   0],
         [ 34,  34,  34],
         [255, 255, 255]],
 
        [[224,  74, 255],
         [130,   0, 128],
         [255, 167, 209],
         ...,
         [255, 255, 255],
         [ 34,  34,  34],
         [255, 255, 255]],
 
        ...,
 
        [[148, 22

In [ ]:
print("test")